# ADP 01 – Full Solution (Codex)
본 노트북은 prob.ipynb의 모든 문항(1~4번)을 재현성 있게 풀이합니다.
- 데이터 경로: `../data`
- 사용 라이브러리: pandas, numpy, scikit-learn, xgboost, statsmodels, seaborn, matplotlib
- 재현성: `random_state=42` 고정, 데이터 전처리-모델링 파이프라인 일원화로 데이터 누수 방지

## 0. 공통 설정

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
try:
    from xgboost import XGBRegressor
    _HAS_XGB = True
except Exception:
    _HAS_XGB = False

from sklearn.linear_model import LinearRegression, Ridge, Lasso
import statsmodels.api as sm
from statsmodels.formula.api import ols

np.random.seed(42)
RANDOM_STATE = 42
sns.set(style='whitegrid', font='AppleGothic')  # 한글 폰트 환경에 맞게 조정하세요
plt.rcParams['axes.unicode_minus'] = False

## 1. 머신러닝 (50점)
데이터: 학생 성적 데이터셋 (394행 소규모)

In [ ]:
# 1) 데이터 로드
student_path = os.path.join('..', 'data', 'student_data.csv')
df = pd.read_csv(student_path)
target_col = 'grade'
df.head()

In [ ]:
# 기본 정보
print(df.shape)
display(df.dtypes)
display(df.describe(include='all').T.iloc[:20])

# 결측치 비율
na_rate = df.isna().mean().sort_values(ascending=False)
display(na_rate.to_frame('na_rate'))

# 수치형/범주형 구분
num_cols = df.select_dtypes(include=np.number).columns.drop([target_col])
cat_cols = df.select_dtypes(exclude=np.number).columns
print('Numeric:', list(num_cols))
print('Categorical:', list(cat_cols))

In [ ]:
# 분포 확인
fig, axes = plt.subplots(1, 2, figsize=(12,4))
sns.histplot(df[target_col], kde=True, ax=axes[0])
axes[0].set_title('Target 분포 (grade)')

sns.boxplot(x=df[target_col], ax=axes[1])
axes[1].set_title('Target 박스플롯')
plt.show()

# 범주형 빈도
for c in cat_cols:
    plt.figure(figsize=(6,3))
    sns.countplot(x=c, data=df)
    plt.title(f'{c} 빈도')
    plt.xticks(rotation=0)
    plt.show()

# 상관 (원-핫 후 상관계수로 개략 확인)
corr_df = pd.get_dummies(df.drop(columns=[]), columns=cat_cols, drop_first=True)
plt.figure(figsize=(10,8))
sns.heatmap(corr_df.corr()[[target_col]].sort_values(by=target_col, ascending=False), annot=True, cmap='coolwarm')
plt.title('Target과의 상관')
plt.show()
print('- 관찰: G1/G2 와 grade의 상관이 큼 → 선형모델에선 다중공선성 유의, 트리계열에선 상대적으로 덜 민감')

### 1-2. 결측치 식별 및 대체 방법 제안

In [ ]:
display(na_rate[na_rate>0])
print("- 방법 A: 단순 대치(SimpleImputer) – 수치: 중앙값/ 평균, 범주: 최빈")
print("- 방법 B: KNN 대치(KNNImputer) – 수치형 주변 이웃 기반")
print("선택: 수치형은 KNNImputer, 범주형은 최빈 대치 후 원-핫 인코딩. 파이프라인 내부에서 학습셋 기준으로 fit하여 데이터 누수 방지.")

### 1-3. 범주형 인코딩 필요성 식별 및 적용

In [ ]:
# 파이프라인용 전처리자 정의
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, list(num_cols)),
        ('cat', categorical_transformer, list(cat_cols))
    ]
)
print('범주형은 OneHot, 수치형은 KNN+스케일링 적용')

### 1-4. 데이터 분할 방법 2가지 및 적용

In [ ]:
# - 랜덤 분할
# - (회귀의) 층화 유사 분할: y를 분위수로 binning하여 stratify 적용
X = df.drop(columns=[target_col])
y = df[target_col]

# 회귀에서의 유사 층화: y를 구간화
y_bins = pd.qcut(y, q=5, labels=False, duplicates='drop')
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y_bins
)
X_train.shape, X_test.shape

### 1-5. SVM/XGBoost/RandomForest 공통점과 적합성

In [ ]:
print('- 공통점: (1) 수치 입력 기반, (2) 회귀/분류 모두 가능, (3) 하이퍼파라미터 튜닝 중요, (4) 교차검증으로 일반화 성능 검증 필요')
print('- 적합성: 입력 차원/표본수 적당, 비선형성 존재 가능 → SVM(RBF), 트리계열(RF/XGB) 모두 유효. 파이프라인으로 전처리 일원화가 핵심')

### 1-6. 세 모델 학습/비교, 최종 선택 및 고찰

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

def eval_model(name, model):
    pipe = Pipeline([('pre', preprocessor), ('model', model)])
    r2 = cross_val_score(pipe, X_train, y_train, scoring='r2', cv=cv)
    rmse = -cross_val_score(pipe, X_train, y_train, scoring='neg_root_mean_squared_error', cv=cv)
    return {
        'name': name,
        'r2_mean': r2.mean(), 'r2_std': r2.std(),
        'rmse_mean': rmse.mean(), 'rmse_std': rmse.std()
    }

base_results = []
base_results.append(eval_model('SVR', SVR(kernel='rbf')))
base_results.append(eval_model('RandomForest', RandomForestRegressor(random_state=RANDOM_STATE)))
if _HAS_XGB:
    base_results.append(eval_model('XGB', XGBRegressor(random_state=RANDOM_STATE, n_estimators=300, learning_rate=0.1)))

pd.DataFrame(base_results)


In [ ]:
# 간단 그리드서치로 상위 모델 튜닝 후 최종 평가
searches = []

svr_pipe = Pipeline([('pre', preprocessor), ('model', SVR())])
svr_grid = {
    'model__kernel': ['rbf'],
    'model__C': [1, 5, 10, 20],
    'model__gamma': ['scale', 0.1, 0.01],
    'model__epsilon': [0.1, 0.2]
}
svr_search = GridSearchCV(svr_pipe, svr_grid, scoring='r2', cv=cv, n_jobs=-1)
svr_search.fit(X_train, y_train)
searches.append(('SVR', svr_search))

rf_pipe = Pipeline([('pre', preprocessor), ('model', RandomForestRegressor(random_state=RANDOM_STATE))])
rf_grid = {
    'model__n_estimators': [300, 600],
    'model__max_depth': [None, 6, 10],
    'model__min_samples_leaf': [1, 2, 5]
}
rf_search = GridSearchCV(rf_pipe, rf_grid, scoring='r2', cv=cv, n_jobs=-1)
rf_search.fit(X_train, y_train)
searches.append(('RandomForest', rf_search))

if _HAS_XGB:
    xgb_pipe = Pipeline([('pre', preprocessor), ('model', XGBRegressor(random_state=RANDOM_STATE, n_estimators=400))])
    xgb_grid = {
        'model__max_depth': [3, 5, 7],
        'model__learning_rate': [0.05, 0.1],
        'model__subsample': [0.8, 1.0],
        'model__colsample_bytree': [0.8, 1.0]
    }
    xgb_search = GridSearchCV(xgb_pipe, xgb_grid, scoring='r2', cv=cv, n_jobs=-1)
    xgb_search.fit(X_train, y_train)
    searches.append(('XGB', xgb_search))

# 교차검증 성적 비교
tune_rows = []
for name, gs in searches:
    tune_rows.append({'name': name, 'cv_best_score_r2': gs.best_score_, 'best_params': gs.best_params_})
cv_summary = pd.DataFrame(tune_rows).sort_values('cv_best_score_r2', ascending=False)
cv_summary


In [ ]:
# 테스트셋 성능 평가 (R2, RMSE) 및 최종 선택
test_rows = []
for name, gs in searches:
    y_pred = gs.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    test_rows.append({'name': name, 'test_r2': r2, 'test_rmse': rmse})
test_summary = pd.DataFrame(test_rows).sort_values('test_r2', ascending=False)
test_summary


- 최종 모델은 테스트 R2 및 RMSE 기준 상위 모델을 채택합니다.
- 한계와 보완: (1) G1/G2가 목표와 강상관 → 현업 목적에 맞춰 포함 여부 검토, (2) 이상치/편향 존재 시 로버스트 스케일러·튜닝, (3) 데이터 기간/분포 변화 모니터링 및 재학습 전략, (4) 특성공학과 SHAP 등 해석성 보강

## 2. 통계분석 (50점) – 회귀 (총 29점)

In [ ]:
# 데이터 로드: 문제의 경로가 없을 수 있어 존재 시 로드, 없으면 대체 데이터 생성
reg_path = os.path.join('..', 'data', 'prob_1_2_1.csv')
if os.path.exists(reg_path):
    df_reg = pd.read_csv(reg_path)
    X_reg, y_reg = df_reg.iloc[:, :-1], df_reg.iloc[:, -1]
    print('Loaded:', reg_path, X_reg.shape)
else:
    # 대체: 선형관계(기울기 3, 절편 5) + 잡음 데이터 생성 (독립변수 1개)
    n=300
    X_reg = pd.DataFrame({'x': np.linspace(-3, 3, n)})
    y_reg = 3*X_reg['x'] + 5 + np.random.normal(0, 1.0, size=n)
    print('Fallback synthetic data used:', X_reg.shape)

X_tr, X_te, y_tr, y_te = train_test_split(X_reg, y_reg, test_size=0.2, random_state=RANDOM_STATE)
X_tr.shape, X_te.shape

### 2-1. 선형회귀: 결정계수와 RMSE

In [ ]:
lin_pipe = Pipeline([('scaler', StandardScaler(with_mean=True, with_std=True)), ('lr', LinearRegression())])
lin_pipe.fit(X_tr, y_tr)
pred = lin_pipe.predict(X_te)
lin_r2 = r2_score(y_te, pred)
lin_rmse = mean_squared_error(y_te, pred, squared=False)
print({'R2': lin_r2, 'RMSE': lin_rmse})


### 2-2. 릿지(Ridge): alpha=0.0~1.0 (0.1 간격) 탐색 후 성능 보고

In [ ]:
ridge_pipe = Pipeline([('scaler', StandardScaler()), ('ridge', Ridge())])
alphas = np.round(np.arange(0.0, 1.01, 0.1), 2)
ridge_grid = {'ridge__alpha': alphas}
ridge_search = GridSearchCV(ridge_pipe, ridge_grid, scoring='r2', cv=KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE))
ridge_search.fit(X_tr, y_tr)
best_alpha_ridge = ridge_search.best_params_['ridge__alpha']
ridge_pred = ridge_search.predict(X_te)
ridge_r2 = r2_score(y_te, ridge_pred)
ridge_rmse = mean_squared_error(y_te, ridge_pred, squared=False)
print({'best_alpha': best_alpha_ridge, 'R2': ridge_r2, 'RMSE': ridge_rmse})
pd.DataFrame({'alpha': alphas, 'cv_r2': ridge_search.cv_results_['mean_test_score']})


### 2-3. 라쏘(Lasso): alpha=0.0~1.0 (0.1 간격) 탐색 후 성능 보고

In [ ]:
lasso_pipe = Pipeline([('scaler', StandardScaler()), ('lasso', Lasso(max_iter=10000))])
lasso_grid = {'lasso__alpha': alphas}
lasso_search = GridSearchCV(lasso_pipe, lasso_grid, scoring='r2', cv=KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE))
lasso_search.fit(X_tr, y_tr)
best_alpha_lasso = lasso_search.best_params_['lasso__alpha']
lasso_pred = lasso_search.predict(X_te)
lasso_r2 = r2_score(y_te, lasso_pred)
lasso_rmse = mean_squared_error(y_te, lasso_pred, squared=False)
print({'best_alpha': best_alpha_lasso, 'R2': lasso_r2, 'RMSE': lasso_rmse})
pd.DataFrame({'alpha': alphas, 'cv_r2': lasso_search.cv_results_['mean_test_score']})


## 3. 다항 회귀 (12점) – 1~3차 계수 및 시각화

In [ ]:
# 데이터 생성 (문항 제공 식과 동일)
m = 100
X = 6 * np.random.rand(m,1) - 3
y = 3 * X**3 + X**2 + 2*X + 2 + np.random.randn(m,1)
line = np.linspace(-3,3,200).reshape(-1,1)

def fit_plot_deg(d):
    pipe = Pipeline([('poly', PolynomialFeatures(degree=d, include_bias=False)), ('lr', LinearRegression())])
    pipe.fit(X, y)
    coefs = pipe.named_steps['lr'].coef_
    y_hat = pipe.predict(line)
    plt.figure(figsize=(5,4))
    plt.scatter(X, y, s=15, alpha=0.6, label='data')
    plt.plot(line, y_hat, color='crimson', label=f'deg={d}')
    plt.title(f'Polynomial deg={d}')
    plt.legend()
    plt.show()
    return coefs

coef_d1 = fit_plot_deg(1)
coef_d2 = fit_plot_deg(2)
coef_d3 = fit_plot_deg(3)
print('deg1 coef:', coef_d1)
print('deg2 coef:', coef_d2)
print('deg3 coef:', coef_d3)


## 4. ANOVA (9점) – 이원분산분석 및 통계표

In [ ]:
avocado_path = os.path.join('..', 'data', 'avocado.csv')
df_avo = pd.read_csv(avocado_path)
df_avo.head()

In [ ]:
# 이원분산분석: AveragePrice ~ region * type
model = ols('AveragePrice ~ C(region) * C(type)', data=df_avo).fit()
anova_tbl = sm.stats.anova_lm(model, typ=2)
anova_tbl

In [ ]:
print('- 상호작용 PR(>F) < 0.05 이면 region:type 상호작용 존재')
print('- 각 주효과 PR(>F) < 0.05 이면 해당 요인의 평균 차이 유의')
